In [3]:
from google.colab import drive

!pip install scikit-survival
!pip install lifelines
!pip install torchtuples

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.0/300.0 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.1/222.1 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.5 MB/s eta 0:00:00
  Attempting uninstall: osqp
    Found existing installation: osqp 1.0.5
    Uninstalling osqp-1.0.5:
      Successfully uninstalled osqp-1.0.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score
from lifelines.plotting import add_at_risk_counts
from lifelines.statistics import proportional_hazard_test
import matplotlib.pyplot as plt
import torch
import torchtuples as tt
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import train_test_split
#from pycox.models import LogisticHazard
#from pycox.evaluation import EvalSurv
import xgboost as xgb

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np

import torch
import numpy as np
import random
import os

def seed_everything(seed=42):
    """
    Seed everything to make all operations in PyTorch deterministic.
    Args:
        seed (int): Seed value to set. Default is 42.
    """
    random.seed(seed)         # Python random module.
    np.random.seed(seed)      # Numpy module.
    os.environ['PYTHONHASHSEED'] = str(seed)  # Set PYTHONHASHSEED env variable at a fixed value

    torch.manual_seed(seed)   # Sets the seed for generating random numbers for all devices (both CPU and CUDA).
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # For multi-GPU setups.

    torch.backends.cudnn.deterministic = True  # Forces to use the deterministic algorithm
    torch.backends.cudnn.benchmark = False     # If True, causes cuDNN to benchmark multiple convolution algorithms and select the fastest.

seed_everything()

In [10]:
seed_everything()

# Import necessary library
from lifelines.datasets import load_gbsg2

# Load the GBSG2 dataset
my_df = load_gbsg2()

# Rename the columns
my_df.rename(columns={
    'horTh': 'Therapy',
    'age': 'Age',
    'menostat': 'Menopause',
    'tsize': 'Size',
    'tgrade': 'Grade',
    'pnodes': 'Nodes',
    'progrec': 'Progesterone',
    'estrec': 'Oestrogen',
    'time': 'Duration',
    'cens': 'Event'
}, inplace=True)

# Display the first few rows of the renamed dataset
my_df.head()

,Therapy,Age,Menopause,Size,Grade,Nodes,Progesterone,Oestrogen,Duration,Event
0,no,70,Post,21,II,3,48,66,1814,1
1,yes,56,Post,12,II,7,61,77,2018,1
2,yes,58,Post,35,II,9,52,271,712,1
3,yes,59,Post,17,II,4,60,29,1807,1
4,no,73,Post,35,II,1,26,65,772,1


In [11]:
seed_everything()

import pandas as pd

# Assuming my_df is already loaded and columns are renamed
my_df['Therapy'] = my_df['Therapy'].map({'no': 0, 'yes': 1})

my_df['Age_45to60'] = ((my_df['Age'] > 44) & (my_df['Age'] < 61)).astype(int)
my_df['Age_G60'] = (my_df['Age'] > 60).astype(int)
my_df.drop('Age', axis=1, inplace=True)

my_df['Menopause'] = my_df['Menopause'].map({'Pre': 0, 'Post': 1})

my_df['Grade_II'] = (my_df['Grade'] == 'II').astype(int)
my_df['Grade_III'] = (my_df['Grade'] == 'III').astype(int)
my_df.drop('Grade', axis=1, inplace=True)

my_df['Nodes_4to9'] = ((my_df['Nodes'] > 3) & (my_df['Nodes'] < 10)).astype(int)
my_df['Nodes_G9'] = (my_df['Nodes'] > 9).astype(int)
my_df.drop('Nodes', axis=1, inplace=True)

my_df['Progesterone_L20'] = (my_df['Progesterone'] < 20).astype(int)
my_df.drop('Progesterone', axis=1, inplace=True)

my_df['Oestrogen_L20'] = (my_df['Oestrogen'] < 20).astype(int)
my_df.drop('Oestrogen', axis=1, inplace=True)

my_df['Size_(20,30]'] = ((my_df['Size'] > 20) & (my_df['Size'] <= 30)).astype(int)
my_df['Size_G30'] = (my_df['Size'] > 30).astype(int)
my_df.drop('Size', axis=1, inplace=True)

Cols = ['Age_45to60', 'Age_G60',
        'Menopause',
        'Size_(20,30]', 'Size_G30',
        'Grade_II', 'Grade_III',
        'Nodes_4to9', 'Nodes_G9',
        'Progesterone_L20', 'Oestrogen_L20',
        'Therapy',
        'Duration',	'Event']

my_df = my_df[Cols]

# Display the first few rows of the transformed dataset
my_df.head()


,Age_45to60,Age_G60,Menopause,"Size_(20,30]",Size_G30,Grade_II,Grade_III,Nodes_4to9,Nodes_G9,Progesterone_L20,Oestrogen_L20,Therapy,Duration,Event
0,0,1,1,1,0,1,0,0,0,0,0,0,1814,1
1,1,0,1,0,0,1,0,1,0,0,0,1,2018,1
2,1,0,1,0,1,1,0,1,0,0,0,1,712,1
3,1,0,1,0,0,1,0,1,0,0,0,1,1807,1
4,0,1,1,0,1,1,0,0,0,0,0,0,772,1


In [12]:
import pandas as pd
import matplotlib.pyplot as plt
from lifelines import CoxPHFitter
import numpy as np

# Define clinically meaningful short descriptions
variable_mapping = {
    'Therapy': 'Therapy',
    'Oestrogen_L20': 'Oest <20 fmol',
    'Progesterone_L20': 'Prog <20 fmol',
    'Nodes_G9': 'Nodes ≥10',
    'Nodes_4to9': 'Nodes 4-9',
    'Grade_III': 'Grade III',
    'Grade_II': 'Grade II',
    'Size_G30': 'Tumor >30mm',
    'Size_(20,30]': 'Tumor 21-30mm',
    'Menopause': 'Menopausal',
    'Age_G60': 'Age >60y',
    'Age_45to60': 'Age 46-60y'
}


# Rename the columns in both datasets using the variable_mapping
my_df.rename(columns=variable_mapping, inplace=True)

# Initialize the Cox Proportional Hazards model
cph_real = CoxPHFitter()

# Fit the CoxPH model on the real data
cph_real.fit(my_df, duration_col='Duration', event_col='Event')

<lifelines.CoxPHFitter: fitted with 686 total observations, 387 right-censored observations>

In [13]:
round(cph_real.summary[['exp(coef)', 'exp(coef) lower 95%', 'exp(coef) upper 95%']], 2)

,exp(coef),exp(coef) lower 95%,exp(coef) upper 95%
covariate,,,
Age 46-60y,0.64,0.45,0.91
Age >60y,0.65,0.41,1.03
Menopausal,1.31,0.94,1.83
Tumor 21-30mm,1.22,0.90,1.65
Tumor >30mm,1.31,0.95,1.79
Grade II,1.74,1.06,2.85
Grade III,1.76,1.02,3.03
Nodes 4-9,1.97,1.51,2.58
Nodes ≥10,3.50,2.58,4.75
